<a href="https://colab.research.google.com/github/mereilly/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubun

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-16 01:52:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.70MB/s    in 0.2s    

2021-10-16 01:52:36 (5.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [8]:
from pyspark import SparkFiles
# chosen reviews from https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()
print (f"Full records count from raw data: {df.count()}")

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames to match tables

In [9]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [10]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id" : "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()
print (f"Customer count: {customers_df.count()}")

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows

Customer count: 1415190


In [11]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()

products_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df.show()
print(f"Product count: {products_df.count()}")

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0764102885|Saint Bernards (B...|
|6041026522|ADOPT ME (I Need ...|
|6041027537|CAUTION Red Color...|
|B00006OAM1|Petstep 560 PetST...|
|B000084E44|Omega Paw Self-Cl...|
|B00008DFTI|GOLDEN CAT COMPAN...|
|B00008DFTP|Purina Pro Plan F...|
|B00008Q378|C.E.T. Flavored T...|
|B00014F7XU|Merrick Thanksgiv...|
|B00014FBRW|Kitty Nest 2 Cat ...|
|B000255O0O|Super Pet Rainbow...|
|B0002563BI|Kent Marine Saltw...|
|B0002563UY|AquaClear Activat...|
|B00025K1HU|  Hikari First Bites|
|B00026068S|Marshall Ferret N...|
|B00028HN7I|Dr. Harvey's Le D...|
|B00028ZLNQ|Lipiderm Gel Cap ...|
|B0002AATJ0|PetSafe Door Medi...|
|B0002APMH4|TetraMarine Marin...|
|B0002APNFA|OSI Spirulina Pel...|
+----------+--------------------+
only showing top 20 rows

Product count: 239341


In [12]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(['review_id', 
                          'product_id', 
                          'customer_id', 
                          'product_parent', 
                          to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df.show()
print(f"Reviews count: {review_id_df.count()}")

+--------------+----------+-----------+--------------+-----------+
|     review_id|product_id|customer_id|product_parent|review_date|
+--------------+----------+-----------+--------------+-----------+
| REAKC26P07MDN|B00Q0K9604|   28794885|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|B00MBW5O9W|   11488901|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|B0084OHUIO|   43214993|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|B001GS71K2|   12835065|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|B004ABH1LG|   26334022|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|B00AX0LFM4|   22283621|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|B00DQFZGZ0|   14469895|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|B00DIRF9US|   50896354|     742358789| 2015-08-31|
| R764DBXGRNECG|B00JRCBFUG|   18440567|     869798483| 2015-08-31|
| RW1853GAT0Z9F|B000L3XYZ4|   50502362|     501118658| 2015-08-31|
|R33GITXNUF1AD4|B00BOEXWFG|   33930128|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|B001HBBQKY|   43534290|     420905252| 2015-08

In [13]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id',
                     'star_rating',
                     'helpful_votes',
                     'total_votes',
                     'vine',
                     'verified_purchase'])

vine_df.show()
print (f"Vine count: {vine_df.count()}")

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [30]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-pets-db.ch7spkcdsndt.us-east-2.rds.amazonaws.com:5432/postgresql"
config = {"user":"root", 
          "password": "drowssap", 
          "driver":"org.postgresql.Driver"}

In [34]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)